## COMCATE code enforcement violation summary  
Accepts standard report of annual code violations, by violation event number keyed by apn ('acctid').  This report may have multiple voilations per event, and this notebook splits those out to be counted.

Scoring the violations will require adding a table mapping violation type to scoring weight.  The current verions just counts all the violations up, as equal.

The bottom of this sheet clusters the violations by point of contact, and writes that out to a file for later use in group calculations.


In [1]:
##
# Set Values to run locally or in google drive
## 
# TODO: Set environment dynamically 
enviro = 'dev'
local_folder = '~/projects/'

In [17]:
####
# Set Latest File Names
####
rentals_fn = 'fixed_up_rental_billing_2020.csv'
# TODO: File does not exist not adding historical  
past_cambridge_apns_fn = 'geocoding/CAN-ref.csv'
sdat_fn = 'SDAT-CAN-ref-202105.csv'
violations_fn = 'violations/Cambridge_MD_Internal_Apps_Case_Report_0505-0603_21.xlsx'

In [18]:
###
# Check that package requirments are satisfied 
###
!pip install leven
!pip install simpledbf
!pip install ngram

In [19]:
import pandas as pd
from simpledbf import Dbf5

if enviro == 'dev':
    # set local data file path 
    path = local_folder + 'cambridge_pita/cambridge_tiered_rental/data/'
else: 
    from pydrive.auth import GoogleAuth
    from pydrive.drive import GoogleDrive
    from google.colab import auth
    # set google drive Data file path 
    path = '/content/drive/My Drive/pita 2021/'
    
    from google.colab import drive
    drive.mount('/content/drive')

## Load the registerd rentals property dataset

In [20]:
rentals = pd.read_csv(path+rentals_fn)

## Load MD SDAT (real property search)
#### Enrich the billing data with columns from prior year(s)

In [21]:
cambridge_apns = pd.read_csv(path+past_cambridge_apns_fn)

In [22]:
sdat = pd.read_csv(path+sdat_fn)
##
# Make sure the acctid is in the right format, and limit the results to just Cambridge
##
# Strip white space 
sdat.acctid = sdat.acctid.apply(lambda x: str(x).strip())

# Merge past apns with new sdat data 
sdat = sdat.merge(cambridge_apns,on='acctid',how='outer',indicator=True)
xtra_apns = sdat.query('_merge == "right_only"')
xtra_categories = list(xtra_apns.CAN_OWNCLASS.unique())
print("There are ",len(xtra_apns), "properties that don't match SDAT in the CAN ref.")
print("None of these types impact the analysys?",xtra_categories)
sdat = sdat.query('_merge == "both"').drop(columns='_merge')

/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (8,11,12,13,14,15,16,29,38,45,49,52,53,77,82,86,104,106,112,113,129,137) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


There are  67 properties that don't match SDAT in the CAN ref.
None of these types impact the analysys? ['UNDEVELOPED_HOUSING', 'COMMERCIAL', 'CITY_OF_CAMBRIDGE', 'OTHER', 'DORCHESTER_COUNTY', 'AGRICULTURE', 'HOUSING']


### Merge rentals and SDAT

In [23]:
print("WARNING:",len(rentals.drop(columns='address').merge(sdat,on='acctid',how='outer',indicator=True).query('_merge == "left_only"')),"didn't resolve!")
df = rentals.drop(columns='address').merge(sdat,on='acctid',how='inner')

In [15]:
df = df.query('acctid > "10"')
df

,acctid,Customer Id,Name,Name 2,Address 2,State,Zip,License Id,License Type,Unnamed: 9,...,assessed_value,address_number,address_unit_id,street_direction,street_name,street_type,premise_address_type_mdp_field_premstyp_sdat_field_24,premise_address_city_mdp_field_premcity_sdat_field_25,premise_address_zip_code_mdp_field_premzip_sdat_field_26,mdp_street_address_mdp_field_address
0,1007113935,RR-07388,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00001,RENTAL,True,...,62333,1920,416,NaN,NaN,BOUNDARY,AVE,CAMBRIDGE,21613.0,416 BOUNDARY AVE
1,1007148038,RR-07981,DAGOSTINO COREY,C/O WRIGHT PROP MANAGEMEMENT,,MD,21643,20-00002,RENTAL,True,...,153433,1900,704,NaN,NaN,CHURCH,ST,CAMBRIDGE,21613.0,704 CHURCH ST
2,1007164289,RR-04889,JAMES INVESTMENTS LLC,,,MD,21613,20-00003,RENTAL,True,...,21433,1917,623,NaN,NaN,DOUGLAS,ST,CAMBRIDGE,21613.0,623 DOUGLAS ST
3,1007164289,RR-01273,KHAREL CHHATRA & SARITA,,,MD,21613,20-01350,RENTAL,True,...,21433,1917,623,NaN,NaN,DOUGLAS,ST,CAMBRIDGE,21613.0,623 DOUGLAS ST
4,1007130538,RR-00375,OTTER LLC,C/O KYLE & LISA MARSHALL,,MD,21613,20-00004,RENTAL,True,...,15667,1924,809,NaN,NaN,PHILLIPS,ST,CAMBRIDGE,21613.0,809 PHILLIPS ST
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1460,1007214987,RR-00563,317 MARYLAND AVE LLC,C/O JANET WHITNEY,,MD,21620,20-01523,RENTAL,True,...,160000,2006,900,NaN,NaN,MARSHY,COV,CAMBRIDGE,21613.0,900 MARSHY COVE
1461,1007165730,RR-03036,WOOLFORD LEROY,,,MD,21623,20-01524,RENTAL,True,...,92933,1910,1000,NaN,NaN,RACE,ST,CAMBRIDGE,21613.0,1000 RACE ST
1462,1007152078,RR-03647,PINKETT LATOYA,,,MD,21673,20-01543,RENTAL,True,...,34400,1930,811,NaN,NaN,PINE,ST,CAMBRIDGE,21613.0,811 PINE ST
1463,1007215096,RR-09665,SHEPHERD VILLAGE II LLC,,,MD,21613,20-01546,RENTAL,True,...,149200,2006,900,NaN,NaN,MARSHY,COV,CAMBRIDGE,21613.0,900 MARSHY COVE


#### Format violations list dates and violations lists
pandas df.explode() is used to make records for each row with seprate violations, but we won't do that until needed to avoid memeory pressure when clustering.

In [ ]:
!cp "drive/My Drive/pita 2021/text_cluster.py" .
from text_cluster import assign_clusters
from text_cluster import text_similarity
from text_cluster import cluster_strings

# violations = pd.read_csv("drive/My Drive/pita 2020/codeviolation_20191208.csv")
violations = pd.read_csv("drive/My Drive/comcate/code_violations-20210609.csv")
violations['Open Date'] = violations.apply(lambda x: pd.to_datetime(x['Open Date']), axis = 1)
violations = violations.rename(columns={'Open Date':'Open_Date'})

In [ ]:
import re

def format_contact(s):
  cinfo = re.sub(r'[,.;:-]','', s).split("\n")
  name = cinfo[0]
  address = ""
  if "Property Owner" in cinfo[0]:
    if len(cinfo) > 2:
      name = cinfo[1].replace("'","").upper()
      address = re.sub(r',* *[A-Z][A-Z] \d\d\d\d\d\'*$',"",cinfo[2]).replace("'","").upper()
    else:
      match = re.search(r' \d+ ',cinfo[1])
      if match:
        tokens = cinfo[1].upper().split(" ")
        numbers = [i for i,token in enumerate(tokens) if re.search(r'\d',token)]
        if len(numbers) > 0:
          name = " ".join(tokens[0:numbers[0]])
          address = " ".join(tokens[numbers[0]:-1])
  else:
    print("BAD RECORD ",s)
  return pd.Series([name, address], index=['Contact_Name','Contact_Address'])


violations[['Contact_Name','Contact_Address']] = violations['Contacts'].apply(format_contact)
violations['address'] = violations.fillna("").apply(lambda x: " ".join([str(x['Street Number']),str(x['Street Direction']),str(x['Street Name']),str(x['Street Type'])]).upper(),axis=1)
violations.APN = violations.apply(lambda x: x.APN.strip(u'\u200b'),axis=1)
violations.address = violations.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

#### fixup account ids

In [ ]:
noacctid = violations.query('APN == ""',engine='python').address.array
found_acctids = {v[0]:v[1] for v in df.query('address in @noacctid')[['address','acctid']].values}
violations.acctid = violations.apply(lambda x: found_acctids.get(x.address,x.APN),axis=1)
violations.drop_duplicates(inplace=True)
violations = violations.query('APN.str.contains("[0-9]+")',engine='python')

In [ ]:
print(len(violations))
violations['vtypes'] = violations.apply(lambda x: x['Violation(s)'].split('\n'),axis=1)
print(len(violations))
v_flat = violations.explode('vtypes')

In [ ]:
#pd.DataFrame(v_flat.vtypes.apply(lambda x: x.split('-')[0].strip()).unique()).to_csv('drive/My Drive/pita 2021/violation_types.csv')
#print(len(v_flat.vtypes.unique()))
#v_flat.vtypes.unique()

In [ ]:
#!mv "/content/drive/My Drive/pita 2021/violation_types (1).csv" "/content/drive/My Drive/pita 2021/violation_types.csv" 
v_categories = pd.read_csv('/content/drive/My Drive/pita 2021/violation_types.csv')
v_lookup = dict(zip(v_categories.viloation_types,v_categories.category))
v_flat['violation_category_text'] = v_flat.vtypes.apply(lambda x: x.split('-')[0].strip())
v_flat['type_0'] = v_flat.apply(lambda x: 1 if v_lookup[x.violation_category_text] < 1 else 0,axis=1)
v_flat['type_1'] = v_flat.apply(lambda x: 1 if v_lookup[x.violation_category_text] == 1 else 0,axis=1)
v_flat['type_2'] = v_flat.apply(lambda x: 1 if v_lookup[x.violation_category_text] == 2 else 0,axis=1)
v_flat['type_3'] = v_flat.apply(lambda x: 1 if v_lookup[x.violation_category_text] == 3 else 0,axis=1)
v_flat['type_4'] = v_flat.apply(lambda x: 1 if v_lookup[x.violation_category_text] == 4 else 0,axis=1)

#### Summarize violations of all types.
Split the multi-line violation descriptions, and total the number of individual issues per tax account id.

In [ ]:
import numpy as np
v_flat.groupby('APN').agg({'type_0':['sum'],'type_1':['sum'],'type_2':['sum'],'type_3':['sum'],'type_4':['sum']})

In [ ]:
# violations['vtypes'] = violations.apply(lambda x: x['Violation(s)'].split('\n'),axis=1)
# v_flat = violations.explode('vtypes')
vsummary = v_flat.groupby('APN')['Violation(s)'].value_counts().reset_index(name='vcount')
vs2 = vsummary.groupby(['APN'])['vcount'].sum().reset_index(name='violations_total')
vs2


####Cluster the violations
Assign clusters by contact name and contact address, split from the Contact column.

In [ ]:
%%time
myquery = violations.query('(Open_Date > "2018-12-31")',engine='python')
names = cluster_strings(myquery.Contact_Name.array,'c_contact',metric='jaccard',threshold=0.1)
addresses = cluster_strings(myquery.Contact_Address.array,'c_address',metric='jaccard',threshold=0.1)

In [ ]:
violations['contact_cluster'] = violations.apply(lambda x: names.get(x.Contact_Name,-1), axis=1)
violations['contactadd_cluster'] = violations.apply(lambda x: addresses.get(x.Contact_Address,-1), axis=1)

#### At this point we have clusters for all the code violations

Some tax accounts are associated with more than one cluster.  This can be due to entry errors in the complaint data, or changes in ownership over the complaint period. The default behavior, used as the fixed behavior here, is to return the least ambiguous dominant cluster id.  What this means practically is if the most often cited cluster == -1 (no cluster), but the second most frequent citation in the period is a valid cluster, that valid cluster is assinged to the tax id.  If more than one valid cluster is cited, the most frequently cited is used.

Some of the violations have no tax acctid, in some cases because of address spelling issues.  Some of these can be backfilled.  The ones we can fix, we do.  Many of the others are just vacant lots which will drop out on the join with properties licenesed for rental.

In [ ]:
def best_cluster(cnumbers):
  if len(cnumbers) > 1 and cnumbers.index[0] < 0:
    return cnumbers.index[1]
  else:
    return cnumbers.index[0]
    
vc_df = pd.DataFrame(violations.groupby(['APN'])['contactadd_cluster'].agg(lambda x:best_cluster(x.value_counts())))#.to_dict()
cadd_c = violations.groupby(['APN'])['contact_cluster'].agg(lambda x:best_cluster(x.value_counts()))
vc_df = vc_df.merge(cadd_c,left_index=True,right_index=True)

# fix tax account id's where possible
vc_df = vc_df.merge(vs2, on='APN',how='inner')
vc_df

### Save result as violation_clusters.csv


In [ ]:
vc_df.to_csv('drive/My Drive/pita 2020/violation_clusters.csv')

In [ ]:
vsample = violations.query('(Open_Date > "2018-12-31")')[['APN','address','contact_cluster','Contact_Name','contactadd_cluster','Contact_Address']]
vsample.APN = vsample.APN.astype(str)
vsample = vsample.rename(columns={'APN':'acctid'})
vsample.acctid = vsample.apply(lambda x: x.acctid.strip(u'\u200b'),axis=1)
vsample.address = vsample.apply(lambda x: re.sub(r' +',' ',x.address).strip(),axis=1)

noacctid = vsample.query('acctid == ""',engine='python').address.array
found_acctids = {v[0]:v[1] for v in df.query('address in @noacctid')[['address','acctid']].values}
vsample.acctid = vsample.apply(lambda x: found_acctids.get(x.address,x.acctid),axis=1)
vsample.query('acctid == ""',engine='python').address.array
#.query('Contact_Name.str.contains("GLG")',engine='python')